In [50]:
from importlib import reload
import subprocess

import pandas as pd
import dask.dataframe as dd
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

from custom_code.metrics import mean_huber, mape, wmape
from custom_code import plotting
from custom_code.settings import PROJECT, BUCKET, RESULTS_DIR, DATA_DIR
from custom_code.download_file_from_gcs import download_file_from_gcs

### LOAD AND PREPARE DATA

In [7]:
RUNTAG = 'ALL_PRODUCTS_ALL_LAGS'

results_df = dd.read_csv('gs://{}/{}/{}_results_with_wa_.csv'.format(BUCKET, RESULTS_DIR, RUNTAG))
results_df = results_df.compute()

In [8]:
product_df = dd.read_csv('gs://{}/{}/product_ALL_PRODUCTS.csv'.format(BUCKET, DATA_DIR))
product_df = product_df.compute()
  
results_df = pd.merge(results_df, product_df, how='left', on='product_id')

In [25]:
RUNTAG = 'OOS_SUBSET_ALL_LAGS'

results_oos_df = dd.read_csv('gs://{}/{}/{}_results_with_wa_.csv'.format(BUCKET, RESULTS_DIR, RUNTAG))
results_oos_df = results_oos_df.compute()

# product_df = dd.read_csv('gs://{}/{}/product_ALL_PRODUCTS.csv'.format(BUCKET, DATA_DIR))
# product_df = product_df.compute()
  
results_oos_df = pd.merge(results_oos_df, product_df, how='left', on='product_id')

In [40]:
RUNTAG = 'PROMO_SUBSET_ALL_LAGS'

results_promo_df = dd.read_csv('gs://{}/{}/{}_results_with_wa_.csv'.format(BUCKET, RESULTS_DIR, RUNTAG))
results_promo_df = results_promo_df.compute()

# product_df = dd.read_csv('gs://{}/{}/product_ALL_PRODUCTS.csv'.format(BUCKET, DATA_DIR))
# product_df = product_df.compute()
  
results_promo_df = pd.merge(results_promo_df, product_df, how='left', on='product_id')

In [51]:
RUNTAG = 'NORMAL_SUBSET_ALL_LAGS'

# file_location = download_file_from_gcs(PROJECT, BUCKET, '{}/{}_results_with_wa_.csv'.format(RESULTS_DIR, RUNTAG))
# results_normal_df = pd.read_csv(file_location)
subprocess.call(['rm', '-f', file_location])
  
results_normal_df = pd.merge(results_normal_df, product_df, how='left', on='product_id')

In [67]:
RUNTAG = 'PRODUCTID_600724_ALL_LAGS'

file_location = download_file_from_gcs(PROJECT, BUCKET, '{}/{}_results_with_wa_.csv'.format(RESULTS_DIR, RUNTAG))
results_oos_productid_df = pd.read_csv(file_location)
subprocess.call(['rm', '-f', file_location])
  
results_oos_productid_df = pd.merge(results_oos_productid_df, product_df, how='left', on='product_id')

In [68]:
RUNTAG = 'PRODUCTID_701608_ALL_LAGS'

file_location = download_file_from_gcs(PROJECT, BUCKET, '{}/{}_results_with_wa_.csv'.format(RESULTS_DIR, RUNTAG))
results_promo_productid_df = pd.read_csv(file_location)
subprocess.call(['rm', '-f', file_location])
  
results_promo_productid_df = pd.merge(results_promo_productid_df, product_df, how='left', on='product_id')

### COMPUTE METRICS

In [9]:
# Create df with OOS after season product types (supplied by Arthur)
results_df_oos = results_df[results_df['product_type_id'].isin([4396, 4999, 5539, 2369, 2703])].dropna()
# Create df with promotion product types (supplied by Arthur)
results_df_promo = results_df[results_df['product_type_id'].isin([2233, 2341, 2627, 5600, 2063])].dropna()
# Create df with normal product types (supplied by Arthur)
results_df_normal = results_df[results_df['product_type_id'].isin([2452, 2458, 2096, 2090,
                                                                   2048, 2250, 2562, 9504])].dropna()

In [11]:
# How many products are in the subsetted categories?
'Products in OOS subset: {}, products in Promo subset: {}, products in Normal subset:{}'.format(results_df_oos['product_id'].nunique(), results_df_promo['product_id'].nunique(), results_df_normal['product_id'].nunique())

'Products in OOS subset: 3874, products in Promo subset: 861, products in Normal subset:10335'

#### OOS AND PROMO METRICS

In [12]:
# OOS types
test_df_oos = results_df_oos[results_df_oos.is_test & results_df_oos.on_stock]
if len(test_df_oos) > 0:
    print('OOS product types overall metrics [LGBM, WA] \n Huber: {:.5} {:.5} \n MSE: {:.5} {:.5} \n MAE: {:.5} {:.5} \n MAPE: {:.5} {:.5} \n wMAPE: {:.5} {:.5}'.format(
        mean_huber(test_df_oos['actual'], test_df_oos['lgbm']),
        mean_huber(test_df_oos['actual'], test_df_oos['wa']),
        mse(test_df_oos['actual'], test_df_oos['lgbm']),
        mse(test_df_oos['actual'], test_df_oos['wa']),
        mae(test_df_oos['actual'], test_df_oos['lgbm']),
        mae(test_df_oos['actual'], test_df_oos['wa']),
        mape(test_df_oos['actual'], test_df_oos['lgbm']),
        mape(test_df_oos['actual'], test_df_oos['wa']),
        wmape(test_df_oos['actual'], test_df_oos['lgbm']),
        wmape(test_df_oos['actual'], test_df_oos['wa'])))

OOS product types overall metrics [LGBM, WA] 
 Huber: 0.13957 0.147 
 MSE: 9.5285 8.2519 
 MAE: 0.22689 0.23839 
 MAPE: 0.096533 0.11462 
 wMAPE: 0.99757 1.0481


In [13]:
# Promo types
test_df_promo = results_df_promo[results_df_promo.is_test & results_df_promo.on_stock]
if len(test_df_promo) > 0:
    print('Promo product types overall metrics [LGBM, WA] \n Huber: {:.5} {:.5} \n MSE: {:.5} {:.5} \n MAE: {:.5} {:.5} \n MAPE: {:.5} {:.5} \n wMAPE: {:.5} {:.5}'.format(
        mean_huber(test_df_promo['actual'], test_df_promo['lgbm']),
        mean_huber(test_df_promo['actual'], test_df_promo['wa']),
        mse(test_df_promo['actual'], test_df_promo['lgbm']),
        mse(test_df_promo['actual'], test_df_promo['wa']),
        mae(test_df_promo['actual'], test_df_promo['lgbm']),
        mae(test_df_promo['actual'], test_df_promo['wa']),
        mape(test_df_promo['actual'], test_df_promo['lgbm']),
        mape(test_df_promo['actual'], test_df_promo['wa']),
        wmape(test_df_promo['actual'], test_df_promo['lgbm']),
        wmape(test_df_promo['actual'], test_df_promo['wa'])))

Promo product types overall metrics [LGBM, WA] 
 Huber: 1.1642 1.1031 
 MSE: 19.198 15.804 
 MAE: 1.4785 1.4242 
 MAPE: 0.39786 0.45098 
 wMAPE: 0.58198 0.56062


In [15]:
# Normal types
test_df_normal = results_df_normal[results_df_normal.is_test & results_df_normal.on_stock]
if len(test_df_normal) > 0:
    print('Promo product types overall metrics [LGBM, WA] \n Huber: {:.5} {:.5} \n MSE: {:.5} {:.5} \n MAE: {:.5} {:.5} \n MAPE: {:.5} {:.5} \n wMAPE: {:.5} {:.5}'.format(
        mean_huber(test_df_normal['actual'], test_df_normal['lgbm']),
        mean_huber(test_df_normal['actual'], test_df_normal['wa']),
        mse(test_df_normal['actual'], test_df_normal['lgbm']),
        mse(test_df_normal['actual'], test_df_normal['wa']),
        mae(test_df_normal['actual'], test_df_normal['lgbm']),
        mae(test_df_normal['actual'], test_df_normal['wa']),
        mape(test_df_normal['actual'], test_df_normal['lgbm']),
        mape(test_df_normal['actual'], test_df_normal['wa']),
        wmape(test_df_normal['actual'], test_df_normal['lgbm']),
        wmape(test_df_normal['actual'], test_df_normal['wa'])))

Promo product types overall metrics [LGBM, WA] 
 Huber: 0.13282 0.13422 
 MSE: 0.59723 0.55996 
 MAE: 0.26378 0.26324 
 MAPE: 0.14581 0.15346 
 wMAPE: 0.92286 0.92098


#### PRODUCT SPECIFIC METRICS

In [16]:
# Calculate metrics per product_id for all products (takes a couple mins to compute all metrics)
test_df_grouped = results_df[results_df.is_test & results_df.on_stock].dropna().groupby('product_id')
test_df_grouped_metrics = pd.DataFrame()  # Create placeholder df
test_df_grouped_metrics['huber_lgbm'] = test_df_grouped.apply(lambda x: mean_huber(x['actual'], x['lgbm']))
test_df_grouped_metrics['huber_wa'] = test_df_grouped.apply(lambda x: mean_huber(x['actual'], x['wa']))
test_df_grouped_metrics['mse_lgbm'] = test_df_grouped.apply(lambda x: mse(x['actual'], x['lgbm']))
test_df_grouped_metrics['mse_wa'] = test_df_grouped.apply(lambda x: mse(x['actual'], x['wa']))
test_df_grouped_metrics['mae_lgbm'] = test_df_grouped.apply(lambda x: mae(x['actual'], x['lgbm']))
test_df_grouped_metrics['mae_wa'] = test_df_grouped.apply(lambda x: mae(x['actual'], x['wa']))
test_df_grouped_metrics['mape_lgbm'] = test_df_grouped.apply(lambda x: mape(x['actual'], x['lgbm']))
test_df_grouped_metrics['mape_wa'] = test_df_grouped.apply(lambda x: mape(x['actual'], x['wa']))
test_df_grouped_metrics['wmape_lgbm'] = test_df_grouped.apply(lambda x: wmape(x['actual'], x['lgbm']))
test_df_grouped_metrics['wmape_wa'] = test_df_grouped.apply(lambda x: wmape(x['actual'], x['wa']))
test_df_grouped_metrics = test_df_grouped_metrics.reset_index()

# Calculate differences between LightGBM and WA predictions
# Form is LightGBM_error - WA_error, such that that a negative error indicates LightGBM outperforms WA
test_df_grouped_metrics['huber_diff'] = test_df_grouped_metrics['huber_lgbm'] - test_df_grouped_metrics['huber_wa']
test_df_grouped_metrics['mae_diff'] = test_df_grouped_metrics['mae_lgbm'] - test_df_grouped_metrics['mae_wa']
test_df_grouped_metrics['mse_diff'] = test_df_grouped_metrics['mse_lgbm'] - test_df_grouped_metrics['mse_wa']
test_df_grouped_metrics['mape_diff'] = test_df_grouped_metrics['mape_lgbm'] - test_df_grouped_metrics['mape_wa']
test_df_grouped_metrics['wmape_diff'] = test_df_grouped_metrics['wmape_lgbm'] - test_df_grouped_metrics['wmape_wa']

In [17]:
# Find bad/good performing products in all products based on a metric of choice
test_df_grouped_metrics.sort_values('huber_diff', ascending=True).head(10)

,product_id,huber_lgbm,huber_wa,mse_lgbm,mse_wa,mae_lgbm,mae_wa,mape_lgbm,mape_wa,wmape_lgbm,wmape_wa,huber_diff,mae_diff,mse_diff,mape_diff,wmape_diff
11907,600724,47.321084,56.165957,14704.905309,12576.481675,47.805282,56.656104,1.308018,5.111679,0.876178,1.038396,-8.844873,-8.850822,2128.423634,-3.803661,-0.162218
34203,767159,21.598998,29.445535,4986.778788,4580.835666,22.042937,29.928537,1.049886,3.892044,0.980437,1.331177,-7.846537,-7.885600,405.943123,-2.842157,-0.350740
45400,787285,14.189099,21.297123,1322.656508,1330.720628,14.628160,21.797123,1.113894,6.067281,1.068724,1.592484,-7.108024,-7.168963,-8.064120,-4.953386,-0.523760
41641,780840,27.511316,33.965741,3116.744387,2970.657441,28.011165,34.459388,2.166619,5.149132,1.078538,1.326819,-6.454424,-6.448223,146.086945,-2.982513,-0.248281
2911,222837,14.983299,20.246374,434.342591,626.025155,15.474952,20.740890,0.503655,0.655983,0.240831,0.322783,-5.263075,-5.265937,-191.682563,-0.152328,-0.081952
24660,735261,9.133788,14.037301,724.546172,708.530335,9.580468,14.525806,0.722189,3.090325,0.900728,1.365674,-4.903513,-4.945338,16.015837,-2.368136,-0.464946
7422,469667,17.040020,21.740428,1926.875747,1711.795585,17.516107,22.220147,1.121828,4.236204,0.988332,1.253754,-4.700408,-4.704040,215.080161,-3.114376,-0.265422
11460,589948,21.242566,25.905523,5401.785899,4441.917568,21.683096,26.362379,1.046944,4.324533,0.984663,1.197156,-4.662957,-4.679283,959.868331,-3.277590,-0.212493
19064,708189,18.438122,22.826395,2481.380629,2245.350536,18.899247,23.305930,1.057040,3.732042,0.929615,1.146371,-4.388272,-4.406683,236.030093,-2.675003,-0.216756
27286,747216,5.394505,9.739506,137.602710,160.495422,5.829870,10.239506,0.938440,3.068643,0.749555,1.316508,-4.345002,-4.409636,-22.892711,-2.130203,-0.566953


#### PRODUCT OOS/PROMO SPECIFIC METRICS

In [18]:
# Filter product metrics of OOS and Promo types
test_df_grouped_metrics_pt = pd.merge(test_df_grouped_metrics, product_df, how='left', on='product_id')
test_df_grouped_metrics_oos = test_df_grouped_metrics_pt[test_df_grouped_metrics_pt['product_type_id'].isin([4396, 4999, 5539, 2369, 2703])]
test_df_grouped_metrics_promo = test_df_grouped_metrics_pt[test_df_grouped_metrics_pt['product_type_id'].isin([2233, 2341, 2627, 5600, 2063])]
test_df_grouped_metrics_normal = test_df_grouped_metrics_pt[test_df_grouped_metrics_pt['product_type_id'].isin([2452, 2458, 2096, 2090, 2048, 2250, 2562, 9504])]

In [55]:
# Find bad/good performing products within subsetted products based on a metric of choice
test_df_grouped_metrics_oos.sort_values('huber_diff', ascending=True).head(5)

,product_id,huber_lgbm,huber_wa,mse_lgbm,mse_wa,mae_lgbm,mae_wa,mape_lgbm,mape_wa,wmape_lgbm,...,mae_diff,mse_diff,mape_diff,wmape_diff,product_type_id,brand_id,manufacturer_id,product_group_id,team_id,subproduct_type_id
11907,600724,47.321084,56.165957,14704.905309,12576.481675,47.805282,56.656104,1.308018,5.111679,0.876178,...,-8.850822,2128.423634,-3.803661,-0.162218,5539,2084,4788,111455,10046,1123
34203,767159,21.598998,29.445535,4986.778788,4580.835666,22.042937,29.928537,1.049886,3.892044,0.980437,...,-7.885600,405.943123,-2.842157,-0.350740,5539,2084,4788,111455,10046,1123
7422,469667,17.040020,21.740428,1926.875747,1711.795585,17.516107,22.220147,1.121828,4.236204,0.988332,...,-4.704040,215.080161,-3.114376,-0.265422,5539,10604,12495,111455,10046,1123
11460,589948,21.242566,25.905523,5401.785899,4441.917568,21.683096,26.362379,1.046944,4.324533,0.984663,...,-4.679283,959.868331,-3.277590,-0.212493,5539,2084,4788,111455,10046,1123
19064,708189,18.438122,22.826395,2481.380629,2245.350536,18.899247,23.305930,1.057040,3.732042,0.929615,...,-4.406683,236.030093,-2.675003,-0.216756,5539,2084,4788,111455,10046,1123


### PLOTTING

In [82]:
reload(plotting)
PRODUCT_ID = 600724
plotting.plot_product(PRODUCT_ID, results_df, test_df_grouped_metrics, fold=0, results_subset_df=results_oos_df, results_productid_df=results_oos_productid_df)

In [57]:
# Find bad/good performing products within subsetted products based on a metric of choice
test_df_grouped_metrics_promo.sort_values('huber_diff', ascending=True).head(5)

,product_id,huber_lgbm,huber_wa,mse_lgbm,mse_wa,mae_lgbm,mae_wa,mape_lgbm,mape_wa,wmape_lgbm,...,mae_diff,mse_diff,mape_diff,wmape_diff,product_type_id,brand_id,manufacturer_id,product_group_id,team_id,subproduct_type_id
18122,701608,0.911786,2.409524,1.993140,8.465328,1.411786,2.909524,0.705893,1.454762,0.705893,...,-1.497738,-6.472188,-0.748869,-0.748869,2627,6,89,164502,2044,2128
4992,360926,12.826528,13.966796,686.924958,597.875117,13.312713,14.456646,0.425062,0.550033,0.363529,...,-1.143933,89.049841,-0.124971,-0.031237,2063,902,3722,70511,10045,390
4991,360908,13.168749,13.912097,422.183725,409.712076,13.657626,14.403839,0.686255,0.727696,0.311299,...,-0.746213,12.471649,-0.041441,-0.017008,2063,902,3722,70511,10045,390
7299,466528,1.444672,2.031746,3.781748,6.409739,1.944672,2.531746,1.944672,2.531746,1.944672,...,-0.587075,-2.627991,-0.587075,-0.587075,2063,252,603,70511,10045,390
7230,464759,2.435662,2.906746,9.009633,12.044474,2.935662,3.406746,1.963732,2.245635,2.935662,...,-0.471084,-3.034841,-0.281903,-0.471084,2627,37,81,164502,2044,2127


In [81]:
reload(plotting)
PRODUCT_ID = 701608
plotting.plot_product(PRODUCT_ID, results_df, test_df_grouped_metrics, fold=6, results_subset_df=results_promo_df, results_productid_df=results_promo_productid_df)

In [78]:
# Find bad/good performing products within subsetted products based on a metric of choice
test_df_grouped_metrics_normal.sort_values('huber_diff', ascending=True).head(10)

,product_id,huber_lgbm,huber_wa,mse_lgbm,mse_wa,mae_lgbm,mae_wa,mape_lgbm,mape_wa,wmape_lgbm,...,mae_diff,mse_diff,mape_diff,wmape_diff,product_type_id,brand_id,manufacturer_id,product_group_id,team_id,subproduct_type_id
8682,516976,0.316652,1.875397,0.633305,5.642510,0.795804,2.375397,0.795804,2.375397,0.795804,...,-1.579592,-5.009205,-1.579592,-1.579592,2090,10403,12394,162505,1,388
37291,772866,0.544381,1.380526,1.270244,4.727821,0.954398,1.854233,0.944955,1.810736,8.589583,...,-0.899835,-3.457577,-0.865781,-8.098512,2458,17726,19717,162505,1,365
39012,775738,1.541020,2.349628,13.957216,16.631230,1.964123,2.833900,0.690210,1.134901,0.824931,...,-0.869778,-2.674014,-0.444690,-0.365307,2458,58,421,162505,1,365
31087,759018,3.672313,4.192063,17.408198,22.015458,4.172313,4.692063,4.172313,4.692063,4.172313,...,-0.519750,-4.607261,-0.519750,-0.519750,2090,6218,8673,162505,1,388
42817,782831,3.107960,3.581861,28.508190,31.802187,3.566120,4.045894,0.805662,1.114824,0.609287,...,-0.479774,-3.293997,-0.309162,-0.081971,2458,37,81,162505,1,365
48039,794086,4.382633,4.800356,51.270917,52.719919,4.858804,5.280495,0.495499,0.575823,0.367988,...,-0.421691,-1.449002,-0.080325,-0.031937,2458,32,5,208516,8045,365
44730,786040,2.118033,2.472018,14.666220,15.346360,2.567637,2.951124,0.710494,1.067350,0.688831,...,-0.383487,-0.680139,-0.356857,-0.102880,2458,6218,8673,162505,1,365
39011,775737,1.155844,1.468033,5.105443,6.610278,1.598619,1.927857,0.930063,1.148668,1.065746,...,-0.329238,-1.504835,-0.218606,-0.219492,2458,58,421,162505,1,365
38257,774568,0.848677,1.153016,1.869962,2.969866,1.345186,1.652976,1.345186,1.652976,10.761491,...,-0.307790,-1.099903,-0.307790,-2.462318,2458,1488,4263,162505,1,365
44689,785998,1.194445,1.484043,7.253822,7.516996,1.590767,1.906411,0.579862,0.799312,0.673017,...,-0.315644,-0.263174,-0.219451,-0.133542,2458,6218,8673,162505,1,365


In [79]:
reload(plotting)
PRODUCT_ID = 785998
plotting.plot_product(PRODUCT_ID, results_df, test_df_grouped_metrics, fold=0, results_subset_df=results_normal_df)